not finished

In [1]:
import sys, json
from datetime import datetime as dt
import numpy as np
import scipy
from preconditioners.utils import generate_c, generate_centered_gaussian_data
from preconditioners.variance_cov_approx import *
from sklearn.covariance import  GraphicalLasso

In [2]:
# parameters of the run
n_epochs = 100
iter_per_epoch = 500
tol = 0.00001
n = 50
d = 150
simga_2 = 1
ro = 0.5
regime = 'autoregressive'
regul_lambda = 0 # for log barrier use 0.001
lr_start = 0.5
lr_decay = 0.98

params = {
    'n_epochs' : n_epochs,
    'iter_per_epoch' : iter_per_epoch,
    'tol' : tol,
    'n' : n,
    'd' : d,
    'sigma_2' : simga_2,
    'ro' : ro,
    'regime' : regime,
    'regul_lambda' : regul_lambda,
    'lr_start' : lr_start,
    'lr_decay' : lr_decay
}

In [3]:
# generate data and initialization
c = generate_c(ro=ro,
                regime=regime,
                n=n,
                d=d
                )
w_star = np.random.multivariate_normal(mean=np.zeros(d), cov=np.eye(d))
X, y, xi = generate_centered_gaussian_data(w_star,
                                            c,
                                            n=n,
                                            d=d,
                                            sigma2=simga_2,
                                            fix_norm_of_x=False)

# initialize C (cholesky), cov_inv, regul_lambda and learning rate
# is this a good way to initialize?
cov_empir = X.T.dot(X) / n
cov_inv = np.linalg.inv(cov_empir + 0.1 * np.eye(d))
C = scipy.linalg.cholesky(cov_inv) + 0.5 * generate_c(ro=0.2,
                                                        regime='autoregressive',
                                                        n=n,
                                                        d=d,
                                                        )

/Users/eduardoravkin/Dropbox/My Mac (Eduard’s MacBook Pro)/Desktop/ml_research/PhD/preconditioners/src/preconditioners/utils.py:219: UserWarning: Warning, norms of datapoints are not sqrt(d)
  warnings.warn('Warning, norms of datapoints are not sqrt(d)')


In [4]:
"""
Sample code automatically generated on 2022-03-14 17:09:14

by geno from www.geno-project.org

from input

parameters
  matrix X
variables
  matrix C
min
  tr(inv(X*C*C'*X'))
st
  tr(C*C') == 250


The generated code is provided "as is" without warranty of any kind.
"""

from __future__ import division, print_function, absolute_import

from math import inf
from timeit import default_timer as timer
try:
    from genosolver import minimize, check_version
    USE_GENO_SOLVER = True
except ImportError:
    from scipy.optimize import minimize
    USE_GENO_SOLVER = False
    WRN = 'WARNING: GENO solver not installed. Using SciPy solver instead.\n' + \
          'Run:     pip install genosolver'
    print('*' * 63)
    print(WRN)
    print('*' * 63)



class GenoNLP:
    def __init__(self, X, CInit, np):
        self.np = np
        self.X = X
        self.CInit = CInit
        assert isinstance(X, self.np.ndarray)
        dim = X.shape
        assert len(dim) == 2
        self.X_rows = dim[0]
        self.X_cols = dim[1]
        assert isinstance(CInit, self.np.ndarray)
        dim = CInit.shape
        assert len(dim) == 2
        self.C_rows = dim[0]
        self.C_cols = dim[1]
        self.C_size = self.C_rows * self.C_cols
        # the following dim assertions need to hold for this problem
        assert self.X_cols == self.C_rows
        assert self.X_cols == self.C_rows

    def getLowerBounds(self):
        bounds = []
        bounds += [-inf] * self.C_size
        return self.np.array(bounds)

    def getUpperBounds(self):
        bounds = []
        bounds += [inf] * self.C_size
        return self.np.array(bounds)

    def getStartingPoint(self):
        return self.CInit.reshape(-1)

    def variables(self, _x):
        C = _x
        C = C.reshape(self.C_rows, self.C_cols)
        return C

    def fAndG(self, _x):
        C = self.variables(_x)
        T_0 = self.np.linalg.inv((((self.X).dot(C)).dot(C.T)).dot(self.X.T))
        f_ = self.np.trace(T_0)
        g_0 = -(2 * ((((self.X.T).dot(T_0)).dot(T_0)).dot(self.X)).dot(C))
        g_ = g_0.reshape(-1)
        return f_, g_

    def functionValueEqConstraint000(self, _x):
        C = self.variables(_x)
        f = (self.np.trace((C).dot(C.T)) - 250)
        return f

    def gradientEqConstraint000(self, _x):
        C = self.variables(_x)
        g_ = ((2 * C)).reshape(-1)
        return g_

    def jacProdEqConstraint000(self, _x, _v):
        C = self.variables(_x)
        gv_ = (((2 * _v) * C)).reshape(-1)
        return gv_

def solve(X, CInit, np):
    start = timer()
    NLP = GenoNLP(X, CInit, np)
    x0 = NLP.getStartingPoint()
    lb = NLP.getLowerBounds()
    ub = NLP.getUpperBounds()
    # These are the standard solver options, they can be omitted.
    options = {'eps_pg' : 1E-4,
               'constraint_tol' : 1E-4,
               'max_iter' : 3000,
               'm' : 10,
               'ls' : 0,
               'verbose' : 5  # Set it to 0 to fully mute it.
              }

    if USE_GENO_SOLVER:
        # Check if installed GENO solver version is sufficient.
        check_version('0.1.0')
        constraints = ({'type' : 'eq',
                        'fun' : NLP.functionValueEqConstraint000,
                        'jacprod' : NLP.jacProdEqConstraint000})
        result = minimize(NLP.fAndG, x0, lb=lb, ub=ub, options=options,
                      constraints=constraints, np=np)
    else:
        constraints = ({'type' : 'eq',
                        'fun' : NLP.functionValueEqConstraint000,
                        'jac' : NLP.gradientEqConstraint000})
        result = minimize(NLP.fAndG, x0, jac=True, method='SLSQP',
                          bounds=list(zip(lb, ub)),
                          constraints=constraints)

    # assemble solution and map back to original problem
    C = NLP.variables(result.x)
    elapsed = timer() - start
    print('solving took %.3f sec' % elapsed)
    return result, C

def generateRandomData(np):
    np.random.seed(0)
    X = np.random.randn(3, 3)
    CInit = np.random.randn(3, 3)
    return X, CInit


import numpy as np
# import cupy as np  # uncomment this for GPU usage
print('solving ...')
result, C = solve(X, CInit = C, np=np)

In [104]:
c

array([[1.00000000e+00, 5.00000000e-01, 2.50000000e-01, ...,
        5.60519386e-45, 2.80259693e-45, 1.40129846e-45],
       [5.00000000e-01, 1.00000000e+00, 5.00000000e-01, ...,
        1.12103877e-44, 5.60519386e-45, 2.80259693e-45],
       [2.50000000e-01, 5.00000000e-01, 1.00000000e+00, ...,
        2.24207754e-44, 1.12103877e-44, 5.60519386e-45],
       ...,
       [5.60519386e-45, 1.12103877e-44, 2.24207754e-44, ...,
        1.00000000e+00, 5.00000000e-01, 2.50000000e-01],
       [2.80259693e-45, 5.60519386e-45, 1.12103877e-44, ...,
        5.00000000e-01, 1.00000000e+00, 5.00000000e-01],
       [1.40129846e-45, 2.80259693e-45, 5.60519386e-45, ...,
        2.50000000e-01, 5.00000000e-01, 1.00000000e+00]])

In [105]:
np.linalg.inv(c)

array([[ 1.33333333e+000, -6.66666667e-001, -5.55111512e-017, ...,
         4.14867685e-061,  2.07433843e-061, -1.03716921e-061],
       [-6.66666667e-001,  1.66666667e+000, -6.66666667e-001, ...,
        -5.76344236e-224, -2.88172118e-224,  5.76344236e-224],
       [ 0.00000000e+000, -6.66666667e-001,  1.66666667e+000, ...,
        -2.07649895e-207, -1.03824947e-207,  2.07649895e-207],
       ...,
       [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000, ...,
         1.66666667e+000, -6.66666667e-001,  1.85037171e-017],
       [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000, ...,
        -6.66666667e-001,  1.66666667e+000, -6.66666667e-001],
       [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000, ...,
         0.00000000e+000, -6.66666667e-001,  1.33333333e+000]])

In [106]:
C.dot(C.T)

array([[ 9.89520193e+00, -3.92320164e-01,  2.29962207e-01, ...,
         6.17563556e-01,  2.78341444e-01, -4.95977851e-01],
       [-3.92320164e-01,  1.03365965e+01, -6.09590015e-01, ...,
        -6.93417536e-02,  2.73705724e-02, -3.28900260e-01],
       [ 2.29962207e-01, -6.09590015e-01,  1.04415231e+01, ...,
         1.24231820e-01, -2.04036753e-01,  7.22706262e-03],
       ...,
       [ 6.17563556e-01, -6.93417536e-02,  1.24231820e-01, ...,
         5.66291624e+00,  3.57854622e-01,  9.58126172e-02],
       [ 2.78341444e-01,  2.73705724e-02, -2.04036753e-01, ...,
         3.57854622e-01,  5.10210840e+00,  5.18076710e-01],
       [-4.95977851e-01, -3.28900260e-01,  7.22706262e-03, ...,
         9.58126172e-02,  5.18076710e-01,  5.51519680e+00]])

In [107]:
np.linalg.inv(C.dot(C.T))

array([[ 1.22583942e-01,  9.36113205e-03,  6.84443450e-04, ...,
        -2.32679769e-02, -3.90986181e-03,  1.60182080e-02],
       [ 9.36113205e-03,  1.19075609e-01,  1.28861926e-02, ...,
         5.41165173e-03, -5.90506178e-05,  6.88490694e-03],
       [ 6.84443450e-04,  1.28861926e-02,  1.19912803e-01, ...,
        -6.13828992e-03,  1.25014925e-02, -2.11518247e-03],
       ...,
       [-2.32679769e-02,  5.41165173e-03, -6.13828992e-03, ...,
         2.48439364e-01, -3.82721064e-02,  4.80840389e-04],
       [-3.90986181e-03, -5.90506178e-05,  1.25014925e-02, ...,
        -3.82721064e-02,  3.39630761e-01, -5.35809042e-02],
       [ 1.60182080e-02,  6.88490694e-03, -2.11518247e-03, ...,
         4.80840389e-04, -5.35809042e-02,  2.71021784e-01]])

In [108]:
cov_empir

array([[ 0.9030155 ,  0.43696357,  0.19571924, ...,  0.06978713,
         0.06980953,  0.02296126],
       [ 0.43696357,  0.99124956,  0.61667035, ...,  0.07869875,
         0.02209223,  0.07085664],
       [ 0.19571924,  0.61667035,  0.96804366, ...,  0.10616198,
        -0.09106277,  0.02197274],
       ...,
       [ 0.06978713,  0.07869875,  0.10616198, ...,  1.01270856,
         0.5186546 ,  0.38425201],
       [ 0.06980953,  0.02209223, -0.09106277, ...,  0.5186546 ,
         0.98808604,  0.51923817],
       [ 0.02296126,  0.07085664,  0.02197274, ...,  0.38425201,
         0.51923817,  1.05212325]])

In [109]:
gl = GraphicalLasso(assume_centered=True, alpha=0.25, tol=1e-4).fit(X)

In [110]:
gl.precision_

array([[ 1.2852561 , -0.18191025, -0.        , ..., -0.        ,
        -0.        , -0.        ],
       [-0.18191025,  1.30695299, -0.42002534, ..., -0.        ,
        -0.        , -0.        ],
       [-0.        , -0.42002534,  1.40382108, ..., -0.        ,
         0.        , -0.        ],
       ...,
       [-0.        , -0.        , -0.        , ...,  1.11683749,
        -0.26577359, -0.06358468],
       [-0.        , -0.        ,  0.        , ..., -0.26577359,
         1.23859462, -0.25257571],
       [-0.        , -0.        , -0.        , ..., -0.06358468,
        -0.25257571,  1.08902042]])

In [111]:
print(np.linalg.norm(C.dot(C.T) - np.linalg.inv(c)))
print(np.linalg.norm(C.dot(C.T)/10 - np.linalg.inv(c)))
print(np.linalg.norm(gl.precision_ - np.linalg.inv(c)))
print(np.linalg.norm(np.eye(d) - np.linalg.inv(c)))

101.58320285676734
15.486163684815262
9.677548073635403
14.087031072743628


In [112]:
print(np.linalg.norm(np.diag(C.dot(C.T)/10) - np.diag(np.linalg.inv(c))))
print(np.linalg.norm(np.diag(gl.precision_) - np.diag(np.linalg.inv(c))))
print(np.linalg.norm(np.diag(np.eye(d)) - np.diag(np.linalg.inv(c))))

9.687289015586297
5.619379139424423
8.124038404635959


In [113]:
print(np.diag(gl.precision_)[:10])
print(np.diag(C.dot(C.T)-9)[:10])
print(np.diag(np.linalg.inv(c))[:10])

[1.2852561  1.30695299 1.40382108 1.21949981 1.23904836 1.31463549
 1.2039896  1.02636606 1.33756259 1.47205837]
[0.89520193 1.33659652 1.44152307 1.33496846 1.65222431 1.52922659
 1.75320315 1.05496604 1.41765909 1.74532465]
[1.33333333 1.66666667 1.66666667 1.66666667 1.66666667 1.66666667
 1.66666667 1.66666667 1.66666667 1.66666667]
